<a href="https://colab.research.google.com/github/belovelace/RAG_chatbot/blob/main/MedQA_RAG_Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏥 MedQA 기반 의료 RAG 시스템 - Phase 1

이 노트북은 MedQA 데이터셋을 활용하여 의료 특화 RAG(Retrieval-Augmented Generation) 시스템을 구축합니다.

## Phase 1 목표
1. MedQA 데이터 로딩 및 전처리
2. 기존 벡터 검색에 MedQA 데이터 적용
3. 의료 특화 프롬프트 템플릿 개발

## 1. 필요한 라이브러리 설치 및 가져오기

In [ ]:
# 필요한 라이브러리 설치
!pip install openai tiktoken python-dotenv scipy pandas numpy requests
!pip install datasets  # Hugging Face datasets for MedQA

import os
import re
import json
import time
import ast
import pandas as pd
import numpy as np
import tiktoken
import requests
from openai import OpenAI
from scipy import spatial
from dotenv import load_dotenv
from datasets import load_dataset
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 설치 및 가져오기 완료!")

✅ 라이브러리 설치 및 가져오기 완료!


## 2. OpenAI API 설정

In [ ]:
# OpenAI API 키 설정 (환경변수 또는 직접 입력)
# 보안을 위해 getpass 사용
import getpass

if not os.getenv("OPENAI_API_KEY"):
    api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
    os.environ["OPENAI_API_KEY"] = api_key

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 임베딩 모델 설정
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 1500

tokenizer = tiktoken.get_encoding(embedding_encoding)
print("✅ OpenAI API 설정 완료!")

✅ OpenAI API 설정 완료!


## 3. MedQA 데이터셋 로딩 및 전처리

In [ ]:
dataset = load_dataset("GBaker/MedQA-USMLE-4-options", split="train")

In [ ]:
import pandas as pd
from datasets import load_dataset
import json
import os

def load_medqa_dataset(subset_size: int = 1000) -> pd.DataFrame:
    """
    MedQA 데이터셋을 로딩하고 전처리합니다.
    여러 방법을 순차적으로 시도하여 최대한 실제 데이터를 로딩합니다.

    Args:
        subset_size: 사용할 데이터 개수 (전체 데이터셋이 크므로 일부만 사용)

    Returns:
        pandas.DataFrame: 전처리된 MedQA 데이터
    """

    # 방법 1: trust_remote_code=True로 시도
    try:
        print("🔄 MedQA 데이터셋 다운로드 중 (방법 1: trust_remote_code)...")
        dataset = load_dataset("bigbio/med_qa", "med_qa_en_bigbio_qa",
                             split="train", trust_remote_code=True)
        return process_medqa_dataset(dataset, subset_size)
    except Exception as e:
        print(f"❌ 방법 1 실패: {e}")

    # 방법 2: 다른 MedQA 변형 시도
    medqa_variants = [
        ("GBaker/MedQA-USMLE-4-options", None),
        ("medmcqa", None),
        ("pubmed_qa", "pqa_labeled"),
        ("bigbio/med_qa", "med_qa_en_source")
    ]

    for dataset_name, config in medqa_variants:
        try:
            print(f"🔄 시도 중: {dataset_name}")
            if config:
                dataset = load_dataset(dataset_name, config, split="train")
            else:
                dataset = load_dataset(dataset_name, split="train")

            return process_alternative_dataset(dataset, dataset_name, subset_size)

        except Exception as e:
            print(f"❌ {dataset_name} 실패: {e}")
            continue

    # 방법 3: 로컬 JSONL 파일 직접 로딩 시도
    print("🔄 로컬 JSONL 파일 확인 중...")
    local_files = [
        "phrases_no_exclude_train.jsonl",
        "phrases_no_exclude_test.jsonl",
        "./phrases_no_exclude_train.jsonl",
        "./phrases_no_exclude_test.jsonl"
    ]

    for file_path in local_files:
        if os.path.exists(file_path):
            try:
                print(f"📁 로컬 파일 발견: {file_path}")
                return load_from_jsonl(file_path, subset_size)
            except Exception as e:
                print(f"❌ {file_path} 로딩 실패: {e}")
                continue

    # 방법 4: 온라인에서 직접 다운로드 시도
    print("🔄 온라인 소스에서 직접 다운로드 시도...")
    try:
        return download_medqa_directly(subset_size)
    except Exception as e:
        print(f"❌ 직접 다운로드 실패: {e}")

    # 최종: 모든 방법 실패
    raise Exception("❌ 모든 MedQA 데이터 로딩 방법이 실패했습니다. 네트워크 연결이나 데이터셋 가용성을 확인해주세요.")


def process_medqa_dataset(dataset, subset_size: int) -> pd.DataFrame:
    """원본 MedQA 데이터셋 처리"""
    if len(dataset) > subset_size:
        dataset = dataset.select(range(subset_size))

    print(f"✅ 로딩된 데이터 개수: {len(dataset)}")

    data_list = []
    for idx, item in enumerate(dataset):
        try:
            question_text = item.get('question', '')
            choices = item.get('choices', [])
            answer_idx = item.get('answer', [0])[0] if item.get('answer') else 0

            choices_text = ""
            if choices:
                choices_text = "\n".join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(choices)])

            correct_answer = ""
            if choices and answer_idx < len(choices):
                correct_answer = choices[answer_idx]

            full_text = f"Question: {question_text}\n\nChoices:\n{choices_text}\n\nCorrect Answer: {correct_answer}"
            document_id = item.get('document_id', f'medqa_{idx}')

            data_list.append({
                'id': document_id,
                'title': f"Medical Question {idx+1}",
                'question': question_text,
                'choices': choices_text,
                'correct_answer': correct_answer,
                'full_text': full_text,
                'source': 'MedQA',
                'category': 'Medical Knowledge'
            })

        except Exception as e:
            print(f"⚠️ 데이터 파싱 오류 (인덱스 {idx}): {e}")
            continue

    df = pd.DataFrame(data_list)
    print(f"✅ MedQA 데이터 전처리 완료! 총 {len(df)}개 항목")
    return df


def process_alternative_dataset(dataset, dataset_name: str, subset_size: int) -> pd.DataFrame:
    """대안 데이터셋 처리"""
    if len(dataset) > subset_size:
        dataset = dataset.select(range(subset_size))

    print(f"✅ {dataset_name} 로딩 성공: {len(dataset)}개")

    data_list = []
    for idx, item in enumerate(dataset):
        try:
            # 데이터셋별 구조 처리
            if 'medmcqa' in dataset_name.lower():
                question = str(item.get('question', ''))
                choices = [str(item.get(f'op{chr(97+i)}', '')) for i in range(4)]  # opa, opb, opc, opd
                answer_idx = item.get('cop', 0)

            elif 'pubmed' in dataset_name.lower():
                question = str(item.get('question', ''))
                choices = ['Yes', 'No', 'Maybe']
                answer_idx = item.get('final_decision', 0)

            elif 'gbaker' in dataset_name.lower() or 'usmle' in dataset_name.lower():
                # GBaker/MedQA-USMLE-4-options 전용 처리
                question = str(item.get('question', ''))

                # 선택지 처리 - 다양한 형태 지원
                if 'options' in item:
                    options = item['options']
                    if isinstance(options, dict):
                        choices = [str(options.get(key, '')) for key in sorted(options.keys())]
                    elif isinstance(options, list):
                        choices = [str(choice) for choice in options]
                    else:
                        choices = ['A', 'B', 'C', 'D']
                elif 'choices' in item:
                    choices_raw = item['choices']
                    if isinstance(choices_raw, list):
                        choices = [str(choice) for choice in choices_raw]
                    else:
                        choices = ['A', 'B', 'C', 'D']
                else:
                    # 개별 선택지 필드 확인
                    choices = []
                    for i, option_key in enumerate(['A', 'B', 'C', 'D', 'option_a', 'option_b', 'option_c', 'option_d']):
                        if option_key in item:
                            choices.append(str(item[option_key]))

                    if len(choices) == 0:
                        choices = ['Option A', 'Option B', 'Option C', 'Option D']

                # 정답 인덱스 처리
                answer_idx = item.get('answer', item.get('correct_answer', item.get('answer_idx', 0)))

            else:  # 기타 형식
                question = str(item.get('question', item.get('text', '')))
                choices = item.get('choices', item.get('options', ['A', 'B', 'C', 'D']))
                if isinstance(choices, str):
                    choices = [choices]
                elif not isinstance(choices, list):
                    choices = ['A', 'B', 'C', 'D']
                choices = [str(choice) for choice in choices]
                answer_idx = item.get('answer', item.get('correct_answer', 0))

            # answer_idx 정규화 및 타입 체크
            if isinstance(answer_idx, list):
                answer_idx = answer_idx[0] if answer_idx else 0

            # 문자열을 숫자로 변환 시도
            try:
                if isinstance(answer_idx, str):
                    if answer_idx.isdigit():
                        answer_idx = int(answer_idx)
                    elif answer_idx.upper() in ['A', 'B', 'C', 'D']:
                        answer_idx = ord(answer_idx.upper()) - ord('A')
                    else:
                        answer_idx = 0
                elif not isinstance(answer_idx, int):
                    answer_idx = 0
            except:
                answer_idx = 0

            # 선택지 필터링 (빈 선택지 제거)
            choices = [choice.strip() for choice in choices if choice and str(choice).strip()]

            # 데이터 유효성 검사
            if not question or not question.strip():
                continue

            if len(choices) == 0:
                continue

            # 안전한 인덱스 접근
            if answer_idx >= len(choices) or answer_idx < 0:
                answer_idx = 0

            choices_text = "\n".join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(choices)])
            correct_answer = choices[answer_idx] if choices else "Unknown"

            full_text = f"Question: {question}\n\nChoices:\n{choices_text}\n\nCorrect Answer: {correct_answer}"

            data_list.append({
                'id': f'{dataset_name.replace("/", "_")}_{idx}',
                'title': f"Medical Question {idx+1}",
                'question': question,
                'choices': choices_text,
                'correct_answer': correct_answer,
                'full_text': full_text,
                'source': dataset_name,
                'category': 'Medical Knowledge'
            })

        except Exception as e:
            print(f"⚠️ 데이터 파싱 오류 (인덱스 {idx}): {e}")
            # 디버깅용 상세 정보
            print(f"  - item keys: {list(item.keys()) if hasattr(item, 'keys') else 'No keys'}")
            if hasattr(item, 'keys') and 'answer' in item:
                print(f"  - answer value: {item['answer']} (type: {type(item['answer'])})")
            continue

    df = pd.DataFrame(data_list)
    print(f"✅ {dataset_name} 데이터 전처리 완료! 총 {len(df)}개 항목")

    # 빈 데이터프레임 체크
    if len(df) == 0:
        print(f"⚠️ {dataset_name}에서 유효한 데이터를 추출할 수 없습니다.")
        # 원본 데이터 구조 분석
        if len(dataset) > 0:
            sample_item = dataset[0]
            print(f"📋 샘플 아이템 구조: {list(sample_item.keys())}")
            for key, value in sample_item.items():
                print(f"  - {key}: {type(value)} = {str(value)[:100]}")

    return df


def load_from_jsonl(file_path: str, subset_size: int) -> pd.DataFrame:
    """JSONL 파일에서 데이터 로딩"""
    data_list = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            if idx >= subset_size:
                break

            try:
                item = json.loads(line.strip())

                # JSONL 구조 파싱 (구조에 따라 조정 필요)
                question = item.get('question', item.get('text', ''))
                choices = item.get('choices', item.get('options', []))
                answer = item.get('answer', item.get('correct_answer', ''))

                if isinstance(choices, list):
                    choices_text = "\n".join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(choices)])
                else:
                    choices_text = str(choices)

                full_text = f"Question: {question}\n\nChoices:\n{choices_text}\n\nCorrect Answer: {answer}"

                data_list.append({
                    'id': f'local_{idx}',
                    'title': f"Medical Question {idx+1}",
                    'question': question,
                    'choices': choices_text,
                    'correct_answer': str(answer),
                    'full_text': full_text,
                    'source': 'Local JSONL',
                    'category': 'Medical Knowledge'
                })

            except Exception as e:
                print(f"⚠️ JSONL 파싱 오류 (라인 {idx}): {e}")
                continue

    df = pd.DataFrame(data_list)
    print(f"✅ JSONL 파일 로딩 완료! 총 {len(df)}개 항목")
    return df


def download_medqa_directly(subset_size: int) -> pd.DataFrame:
    """온라인에서 직접 MedQA 데이터 다운로드"""
    import urllib.request
    import tempfile

    # GitHub이나 공개 저장소에서 MedQA 샘플 데이터 다운로드
    urls = [
        "https://raw.githubusercontent.com/jind11/MedQA/master/data/questions/4_options/questions_clean.json",
        "https://raw.githubusercontent.com/jind11/MedQA/master/data/questions/5_options/questions_clean.json"
    ]

    for url in urls:
        try:
            print(f"📥 다운로드 시도: {url}")

            with tempfile.NamedTemporaryFile(mode='w+', delete=False, suffix='.json') as tmp_file:
                urllib.request.urlretrieve(url, tmp_file.name)

                with open(tmp_file.name, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                os.unlink(tmp_file.name)

                # 데이터 처리
                data_list = []
                for idx, item in enumerate(data[:subset_size]):
                    question = item.get('question', '')
                    options = item.get('options', {})
                    answer_idx = item.get('answer_idx', 0)

                    choices = [options.get(key, '') for key in sorted(options.keys())]
                    choices_text = "\n".join([f"{chr(65+i)}) {choice}" for i, choice in enumerate(choices)])
                    correct_answer = choices[answer_idx] if answer_idx < len(choices) else "Unknown"

                    full_text = f"Question: {question}\n\nChoices:\n{choices_text}\n\nCorrect Answer: {correct_answer}"

                    data_list.append({
                        'id': f'direct_{idx}',
                        'title': f"Medical Question {idx+1}",
                        'question': question,
                        'choices': choices_text,
                        'correct_answer': correct_answer,
                        'full_text': full_text,
                        'source': 'MedQA Direct',
                        'category': 'Medical Knowledge'
                    })

                df = pd.DataFrame(data_list)
                print(f"✅ 직접 다운로드 성공! 총 {len(df)}개 항목")
                return df

        except Exception as e:
            print(f"❌ {url} 다운로드 실패: {e}")
            continue

    raise Exception("모든 직접 다운로드 방법 실패")


# 사용 예시
if __name__ == "__main__":
    try:
        # MedQA 데이터 로딩
        medqa_df = load_medqa_dataset(subset_size=500)

        if len(medqa_df) == 0:
            print("❌ 로딩된 데이터가 없습니다.")
            print("📋 모든 데이터 소스에서 파싱 가능한 항목이 없었습니다.")
        else:
            print("\n📊 데이터 샘플:")
            print(medqa_df[['title', 'question', 'correct_answer']].head())
            print(f"\n📈 데이터셋 정보:")
            print(f"- 총 항목 수: {len(medqa_df)}")
            print(f"- 데이터 소스: {medqa_df['source'].value_counts().to_dict()}")
            print(f"- 카테고리: {medqa_df['category'].value_counts().to_dict()}")

    except Exception as e:
        print(f"❌ 데이터 로딩 실패: {e}")
        print("📋 다음을 확인해주세요:")
        print("  1. 인터넷 연결 상태")
        print("  2. Hugging Face datasets 라이브러리 설치")
        print("  3. 로컬 JSONL 파일 존재 여부")
        print("  4. 필요 권한 및 방화벽 설정")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'bigbio/med_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'bigbio/med_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


🔄 MedQA 데이터셋 다운로드 중 (방법 1: trust_remote_code)...
❌ 방법 1 실패: Dataset scripts are no longer supported, but found med_qa.py
🔄 시도 중: GBaker/MedQA-USMLE-4-options
✅ GBaker/MedQA-USMLE-4-options 로딩 성공: 500개
✅ GBaker/MedQA-USMLE-4-options 데이터 전처리 완료! 총 500개 항목

📊 데이터 샘플:
                title                                           question  \
0  Medical Question 1  A 23-year-old pregnant woman at 22 weeks gesta...   
1  Medical Question 2  A 3-month-old baby died suddenly at night whil...   
2  Medical Question 3  A mother brings her 3-week-old infant to the p...   
3  Medical Question 4  A pulmonary autopsy specimen from a 58-year-ol...   
4  Medical Question 5  A 20-year-old woman presents with menorrhagia ...   

                                      correct_answer  
0                                         Ampicillin  
1  Placing the infant in a supine position on a f...  
2       Abnormal migration of ventral pancreatic bud  
3                                    Thromboembolism  
4  

## 4. 데이터 전처리 및 임베딩 생성

In [ ]:
def preprocess_medical_text(text: str) -> str:
    """
    의료 텍스트 전처리
    """
    # 개행 문자 정리
    text = re.sub(r'\n+', ' ', text)
    # 여러 공백을 하나로
    text = re.sub(r' +', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    return text


def get_embedding_with_retry(text: str, model: str = embedding_model, max_retries: int = 3) -> List[float]:
    """
    재시도 로직이 포함된 임베딩 생성 함수
    """
    text = preprocess_medical_text(text)

    for attempt in range(max_retries):
        try:
            response = client.embeddings.create(
                input=[text],
                model=model
            )
            return response.data[0].embedding

        except Exception as e:
            if attempt < max_retries - 1:
                print(f"⚠️ 임베딩 생성 실패 (시도 {attempt + 1}/{max_retries}): {e}")
                time.sleep(2 ** attempt)  # 지수 백오프
            else:
                print(f"❌ 임베딩 생성 최종 실패: {e}")
                raise e


# 텍스트 전처리
print("🔄 텍스트 전처리 중...")
medqa_df['processed_text'] = medqa_df['full_text'].apply(preprocess_medical_text)

# 토큰 수 계산
medqa_df['n_tokens'] = medqa_df['processed_text'].apply(lambda x: len(tokenizer.encode(x)))

print(f" 토큰 수 통계:")
print(f"  - 평균: {medqa_df['n_tokens'].mean():.1f}")
print(f"  - 최대: {medqa_df['n_tokens'].max()}")
print(f"  - 최소: {medqa_df['n_tokens'].min()}")

# 너무 긴 텍스트 필터링 (토큰 제한)
original_count = len(medqa_df)
medqa_df = medqa_df[medqa_df['n_tokens'] <= max_tokens]
filtered_count = len(medqa_df)

if original_count != filtered_count:
    print(f"⚠️ 긴 텍스트 {original_count - filtered_count}개 제거됨")

print(f"✅ 전처리 완료! 최종 데이터 개수: {len(medqa_df)}")

🔄 텍스트 전처리 중...
 토큰 수 통계:
  - 평균: 210.6
  - 최대: 523
  - 최소: 61
✅ 전처리 완료! 최종 데이터 개수: 500


In [ ]:
# 임베딩 생성
print(" 임베딩 생성 중... (시간이 소요될 수 있습니다)")

embeddings = []
failed_indices = []

for idx, text in enumerate(medqa_df['processed_text']):
    try:
        if idx % 10 == 0:  # 진행상황 표시
            print(f"  진행률: {idx}/{len(medqa_df)} ({idx/len(medqa_df)*100:.1f}%)")

        embedding = get_embedding_with_retry(text)
        embeddings.append(embedding)

    except Exception as e:
        print(f" 임베딩 생성 실패 (인덱스 {idx}): {e}")
        failed_indices.append(idx)
        embeddings.append(None)

# 실패한 항목 제거
if failed_indices:
    print(f" {len(failed_indices)}개 항목에서 임베딩 생성 실패")
    valid_mask = [emb is not None for emb in embeddings]
    medqa_df = medqa_df[valid_mask].reset_index(drop=True)
    embeddings = [emb for emb in embeddings if emb is not None]

# 임베딩을 데이터프레임에 추가
medqa_df['embeddings'] = embeddings

print(f" 임베딩 생성 완료! 총 {len(medqa_df)}개 항목")
print(f" 임베딩 차원: {len(embeddings[0]) if embeddings else 'N/A'}")

# 데이터 저장
medqa_df.to_csv('medqa_embeddings.csv', index=False, encoding='utf-8')
print(" 데이터 저장 완료: medqa_embeddings.csv")

 임베딩 생성 중... (시간이 소요될 수 있습니다)
  진행률: 0/500 (0.0%)
  진행률: 10/500 (2.0%)
  진행률: 20/500 (4.0%)
  진행률: 30/500 (6.0%)
  진행률: 40/500 (8.0%)
  진행률: 50/500 (10.0%)
  진행률: 60/500 (12.0%)
  진행률: 70/500 (14.0%)
  진행률: 80/500 (16.0%)
  진행률: 90/500 (18.0%)
  진행률: 100/500 (20.0%)
  진행률: 110/500 (22.0%)
  진행률: 120/500 (24.0%)
  진행률: 130/500 (26.0%)
  진행률: 140/500 (28.0%)
  진행률: 150/500 (30.0%)
  진행률: 160/500 (32.0%)
  진행률: 170/500 (34.0%)
  진행률: 180/500 (36.0%)
  진행률: 190/500 (38.0%)
  진행률: 200/500 (40.0%)
  진행률: 210/500 (42.0%)
  진행률: 220/500 (44.0%)
  진행률: 230/500 (46.0%)
  진행률: 240/500 (48.0%)
  진행률: 250/500 (50.0%)
  진행률: 260/500 (52.0%)
  진행률: 270/500 (54.0%)
  진행률: 280/500 (56.0%)
  진행률: 290/500 (58.0%)
  진행률: 300/500 (60.0%)
  진행률: 310/500 (62.0%)
  진행률: 320/500 (64.0%)
  진행률: 330/500 (66.0%)
  진행률: 340/500 (68.0%)
  진행률: 350/500 (70.0%)
  진행률: 360/500 (72.0%)
  진행률: 370/500 (74.0%)
  진행률: 380/500 (76.0%)
  진행률: 390/500 (78.0%)
  진행률: 400/500 (80.0%)
  진행률: 410/500 (82.0%)
  진행률: 420/500 (84.0

## 5. 의료 특화 검색 엔진 구현

In [ ]:
def to_numpy_embedding(embedding_data) -> np.ndarray:
    """
    다양한 형태의 임베딩 데이터를 numpy 배열로 변환
    """
    if isinstance(embedding_data, np.ndarray):
        return embedding_data.astype(np.float32)
    elif isinstance(embedding_data, list):
        return np.array(embedding_data, dtype=np.float32)
    elif isinstance(embedding_data, str):
        try:
            # JSON 문자열 파싱 시도
            parsed = json.loads(embedding_data)
            return np.array(parsed, dtype=np.float32)
        except json.JSONDecodeError:
            # literal_eval 시도
            parsed = ast.literal_eval(embedding_data)
            return np.array(parsed, dtype=np.float32)
    else:
        return np.array(embedding_data, dtype=np.float32)


def calculate_similarity_scores(query_embedding: np.ndarray,
                               doc_embeddings: List[np.ndarray],
                               metric: str = "cosine") -> List[float]:
    """
    쿼리와 문서들 간의 유사도 계산
    """
    similarity_functions = {
        "cosine": lambda q, d: 1 - spatial.distance.cosine(q, d),
        "euclidean": lambda q, d: 1 / (1 + spatial.distance.euclidean(q, d)),
        "dot_product": lambda q, d: np.dot(q, d)
    }

    similarity_func = similarity_functions.get(metric, similarity_functions["cosine"])

    scores = []
    for doc_emb in doc_embeddings:
        try:
            score = similarity_func(query_embedding, doc_emb)
            scores.append(score)
        except Exception as e:
            print(f"⚠️ 유사도 계산 오류: {e}")
            scores.append(0.0)

    return scores


def medical_search(query: str,
                  df: pd.DataFrame,
                  top_k: int = 5,
                  category_filter: str = None) -> pd.DataFrame:
    """
    의료 특화 검색 함수

    Args:
        query: 검색 질의
        df: 검색할 데이터프레임
        top_k: 반환할 상위 결과 개수
        category_filter: 카테고리 필터 (예: 'Cardiology')

    Returns:
        검색 결과 데이터프레임
    """
    print(f" 의료 검색 수행: '{query}'")

    # 카테고리 필터 적용
    search_df = df.copy()
    if category_filter:
        search_df = search_df[search_df['category'] == category_filter]
        print(f" 카테고리 필터 적용: {category_filter} ({len(search_df)}개 항목)")

    if len(search_df) == 0:
        print("❌ 검색할 데이터가 없습니다.")
        return pd.DataFrame()

    try:
        # 쿼리 임베딩 생성
        query_embedding = get_embedding_with_retry(query)
        query_emb_array = np.array(query_embedding, dtype=np.float32)

        # 문서 임베딩 변환
        doc_embeddings = [to_numpy_embedding(emb) for emb in search_df['embeddings']]

        # 유사도 계산
        similarities = calculate_similarity_scores(query_emb_array, doc_embeddings)

        # 결과 정렬 및 상위 k개 반환
        search_df = search_df.copy()
        search_df['similarity'] = similarities

        results = search_df.nlargest(top_k, 'similarity')

        print(f"✅ 검색 완료! 상위 {len(results)}개 결과 반환")
        return results

    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return pd.DataFrame()


def create_medical_context(search_results: pd.DataFrame, max_length: int = 2000) -> str:
    """
    검색 결과를 바탕으로 의료 컨텍스트 생성
    """
    if len(search_results) == 0:
        return "관련 의료 정보를 찾을 수 없습니다."

    context_parts = []
    current_length = 0

    for idx, row in search_results.iterrows():
        # 각 결과에 대한 정보 구성
        part = f"""[의료 정보 {len(context_parts) + 1}] (신뢰도: {row['similarity']:.3f})
제목: {row['title']}
분야: {row['category']}
내용: {row['processed_text'][:500]}...

"""

        if current_length + len(part) > max_length:
            break

        context_parts.append(part)
        current_length += len(part)

    return "\n".join(context_parts)


print("✅ 의료 특화 검색 엔진 구현 완료!")

✅ 의료 특화 검색 엔진 구현 완료!


## 6. 의료 특화 프롬프트 템플릿 개발

In [ ]:
class MedicalPromptTemplates:
    """
    의료 분야별 특화 프롬프트 템플릿
    """

    @staticmethod
    def get_medical_assistant_prompt(context: str, question: str, query_type: str = "general") -> str:
        """
        의료 AI 어시스턴트용 기본 프롬프트
        """
        base_prompt = f"""당신은 의료 전문 AI 어시스턴트입니다. 아래 의료 지식 베이스를 참고하여 정확하고 신뢰할 수 있는 답변을 제공하세요.

**중요한 지침:**
1. 제공된 의료 정보만을 기반으로 답변하세요
2. 확실하지 않은 정보는 "추가 의료진 상담이 필요합니다"라고 명시하세요
3. 응급 상황이 의심되면 즉시 응급실 방문을 권하세요
4. 개인적인 의료 조언이 아닌 일반적인 의학 정보임을 명시하세요
5. 정보의 출처와 신뢰도를 함께 제시하세요

**의료 지식 베이스:**
{context}

**사용자 질문:** {question}

**답변 형식:**
- 주요 답변: [핵심 의료 정보]
- 근거: [참조한 의료 지식의 출처]
- 주의사항: [추가 주의사항이나 의료진 상담 필요성]
- 면책조항: 이 정보는 일반적인 의학 정보이며, 개인적인 의료 상담을 대체하지 않습니다.

답변:"""

        return base_prompt

    @staticmethod
    def get_diagnosis_support_prompt(context: str, symptoms: str) -> str:
        """
        진단 지원용 프롬프트
        """
        return f"""당신은 의료 진단 지원 AI입니다. 제시된 증상을 바탕으로 가능한 진단과 추가 검사를 제안하세요.

**주의:** 이는 의료진의 판단을 돕는 도구이며, 최종 진단은 반드시 의료진이 내려야 합니다.

**의료 지식 베이스:**
{context}

**환자 증상:** {symptoms}

**분석 항목:**
1. 가능한 진단 (확률 순)
2. 감별 진단
3. 권장 추가 검사
4. 응급도 평가
5. 초기 치료 방향

답변:"""

    @staticmethod
    def get_treatment_recommendation_prompt(context: str, diagnosis: str) -> str:
        """
        치료 권장사항용 프롬프트
        """
        return f"""당신은 의료 치료 권장사항 제공 AI입니다. 주어진 진단에 대한 표준 치료법을 제시하세요.

**의료 지식 베이스:**
{context}

**진단:** {diagnosis}

**치료 권장사항:**
1. 1차 치료법 (First-line treatment)
2. 대안 치료법 (Alternative treatments)
3. 약물 용법·용량
4. 치료 기간 및 모니터링
5. 부작용 및 주의사항
6. 예후 및 추적관찰

**면책조항:** 모든 치료는 환자의 개별 상황을 고려하여 의료진이 최종 결정해야 합니다.

답변:"""

    @staticmethod
    def get_drug_information_prompt(context: str, drug_name: str) -> str:
        """
        약물 정보 제공용 프롬프트
        """
        return f"""당신은 약물 정보 제공 AI입니다. 요청된 약물에 대한 상세 정보를 제공하세요.

**의료 지식 베이스:**
{context}

**약물명:** {drug_name}

**약물 정보:**
1. 일반명 및 상품명
2. 적응증
3. 작용 기전
4. 용법·용량
5. 금기사항
6. 부작용
7. 약물 상호작용
8. 특별 주의사항

**안전 정보:** 모든 약물 사용은 의료진의 처방과 지시에 따라야 합니다.

답변:"""


def generate_medical_response(question: str,
                            df: pd.DataFrame,
                            response_type: str = "general",
                            category_filter: str = None) -> Dict[str, any]:
    """
    의료 질문에 대한 응답 생성

    Args:
        question: 사용자 질문
        df: 의료 데이터 프레임
        response_type: 응답 유형 ('general', 'diagnosis', 'treatment', 'drug')
        category_filter: 카테고리 필터

    Returns:
        응답 정보가 담긴 딕셔너리
    """
    try:
        # 관련 의료 정보 검색
        search_results = medical_search(question, df, top_k=3, category_filter=category_filter)

        if len(search_results) == 0:
            return {
                "answer": "죄송합니다. 관련된 의료 정보를 찾을 수 없습니다. 의료진에게 직접 상담받으시길 권합니다.",
                "sources": [],
                "confidence": 0.0
            }

        # 컨텍스트 생성
        context = create_medical_context(search_results)

        # 프롬프트 선택
        if response_type == "diagnosis":
            prompt = MedicalPromptTemplates.get_diagnosis_support_prompt(context, question)
        elif response_type == "treatment":
            prompt = MedicalPromptTemplates.get_treatment_recommendation_prompt(context, question)
        elif response_type == "drug":
            prompt = MedicalPromptTemplates.get_drug_information_prompt(context, question)
        else:
            prompt = MedicalPromptTemplates.get_medical_assistant_prompt(context, question)

        # GPT 응답 생성
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,  # 의료 정보는 보수적으로
            max_tokens=1500
        )

        answer = response.choices[0].message.content.strip()

        # 출처 정보 구성
        sources = []
        for _, row in search_results.head(3).iterrows():
            sources.append({
                "title": row['title'],
                "category": row['category'],
                "similarity": row['similarity'],
                "source": row['source']
            })

        # 평균 신뢰도 계산
        avg_confidence = search_results['similarity'].mean() if len(search_results) > 0 else 0.0

        return {
            "answer": answer,
            "sources": sources,
            "confidence": avg_confidence,
            "search_results_count": len(search_results)
        }

    except Exception as e:
        print(f"❌ 응답 생성 오류: {e}")
        return {
            "answer": f"오류가 발생했습니다: {e}. 의료진에게 직접 상담받으시길 권합니다.",
            "sources": [],
            "confidence": 0.0
        }


print("✅ 의료 특화 프롬프트 템플릿 개발 완료!")

✅ 의료 특화 프롬프트 템플릿 개발 완료!


## 7. 시스템 테스트 및 데모

In [50]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import time
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

class ImprovedMedicalRAG:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
        개선된 의료 RAG 시스템
        """
        self.model = SentenceTransformer(model_name)

        # 동적으로 의료 용어를 추출하기 위한 패턴들
        self.medical_patterns = {
            'disease_terms': [
                r'(\w*병)', r'(\w*염)', r'(\w*증)', r'(\w*경색)', r'(\w*종)',
                r'(\w*암)', r'(\w*궤양)', r'(\w*부전)', r'(\w*중독)',
                r'(hypertension)', r'(diabetes)', r'(pneumonia)', r'(infection)',
                r'(disease)', r'(syndrome)', r'(disorder)', r'(cancer)'
            ],
            'medication_terms': [
                r'(\w*제)', r'(\w*약)', r'(\w*정)', r'(\w*캡슐)', r'(\w*시럽)',
                r'(\w*억제제)', r'(\w*차단제)', r'(\w*길항제)', r'(\w*계)',
                r'(medication)', r'(drug)', r'(medicine)', r'(therapy)',
                r'(treatment)', r'(inhibitor)', r'(blocker)'
            ],
            'symptom_terms': [
                r'(\w*통)', r'(\w*열)', r'(\w*기침)', r'(\w*호흡)', r'(\w*어지)',
                r'(\w*피로)', r'(\w*부종)', r'(\w*발진)', r'(\w*가려)',
                r'(pain)', r'(fever)', r'(cough)', r'(symptom)', r'(fatigue)'
            ],
            'procedure_terms': [
                r'(\w*술)', r'(\w*검사)', r'(\w*촬영)', r'(\w*측정)',
                r'(\w*치료)', r'(\w*처치)', r'(\w*수술)',
                r'(surgery)', r'(examination)', r'(procedure)', r'(test)', r'(scan)'
            ]
        }

        # 다국어 의료 용어들 (한국어 + 영어)
        self.multilingual_medical_terms = {
            'diagnosis': ['진단', '검진', '소견', '판독', '결과', 'diagnosis', 'examination', 'test', 'result'],
            'treatment': ['치료', '처방', '투여', '복용', '사용', '적용', 'treatment', 'therapy', 'prescription', 'medication'],
            'prevention': ['예방', '방지', '차단', '보호', '관리', 'prevention', 'protect', 'manage'],
            'anatomy': ['심장', '폐', '간', '신장', '뇌', '위', '장', '혈관', 'heart', 'lung', 'liver', 'kidney', 'brain'],
            'vital_signs': ['혈압', '맥박', '체온', '호흡수', '산소포화도', 'blood pressure', 'pulse', 'temperature', 'bp'],
            'lab_values': ['혈당', '콜레스테롤', '헤모글로빈', '백혈구', '혈소판', 'glucose', 'cholesterol', 'hemoglobin'],
            'hypertension': ['고혈압', '혈압', '수축기', '이완기', 'hypertension', 'blood pressure', 'systolic', 'diastolic', 'bp'],
            'diabetes': ['당뇨병', '혈당', '인슐린', 'diabetes', 'glucose', 'insulin', 'blood sugar'],
            'pneumonia': ['폐렴', '항생제', '폐', '기침', 'pneumonia', 'antibiotic', 'lung', 'cough'],
            'first_line': ['1차', '일차', '첫번째', 'first-line', 'primary', 'initial', 'first choice']
        }

    def preprocess_question(self, question: str) -> str:
        """
        질문 전처리 및 의료 용어 정규화
        """
        normalized_question = question.lower()

        # 동의어 처리
        synonyms = {
            '1차 약물': '일차 치료제',
            '주요 증상': '대표적인 증상',
            '부작용': '이상반응',
            '응급처치': '응급치료'
        }

        for original, normalized in synonyms.items():
            normalized_question = normalized_question.replace(original, normalized)

        return normalized_question

    def extract_medical_entities(self, text: str) -> Dict[str, List[str]]:
        """
        텍스트에서 의료 엔티티를 동적으로 추출 (다국어 지원)
        """
        text_lower = text.lower()
        extracted_entities = {}

        for category, patterns in self.medical_patterns.items():
            entities = []
            for pattern in patterns:
                matches = re.findall(pattern, text_lower)
                if matches:
                    if isinstance(matches[0], tuple):
                        entities.extend([match[0] for match in matches if match[0]])
                    else:
                        entities.extend([match for match in matches if match])
            extracted_entities[category] = list(set(entities))

        # 다국어 의료 용어 매칭
        for category, terms in self.multilingual_medical_terms.items():
            matched_terms = [term for term in terms if term in text_lower]
            if matched_terms:
                extracted_entities[f'multilingual_{category}'] = matched_terms

        return extracted_entities

    def calculate_medical_relevance_silent(self, question: str, document: str) -> float:
        """
        의료 관련성 계산 (다국어 지원, 출력 없음)
        """
        question_lower = question.lower()
        document_lower = document.lower()

        # 핵심 의료 키워드들 (한영 혼합)
        all_keywords = []
        for terms in self.multilingual_medical_terms.values():
            all_keywords.extend(terms)

        keyword_matches = 0
        total_keywords = len(set(all_keywords))

        for keyword in set(all_keywords):
            if keyword in question_lower and keyword in document_lower:
                keyword_matches += 1

        keyword_relevance = keyword_matches / total_keywords if total_keywords > 0 else 0

        # 의료 문맥 점수 (다국어)
        medical_context_patterns = [
            'patient', 'treatment', 'diagnosis', 'symptom', 'medication', 'therapy',
            'hospital', 'doctor', 'nurse', 'surgery', 'examination', 'prescription',
            '환자', '치료', '진단', '증상', '약물', '처방', '병원', '의사', '간호', '수술', '검사'
        ]

        question_context = sum(1 for pattern in medical_context_patterns
                             if pattern in question_lower)
        document_context = sum(1 for pattern in medical_context_patterns
                             if pattern in document_lower)

        if question_context > 0 and document_context > 0:
            context_bonus = min(question_context, document_context) / len(medical_context_patterns)
        else:
            context_bonus = 0

        final_score = (keyword_relevance * 0.7) + (context_bonus * 0.3)
        return min(final_score, 1.0)

    def calculate_medical_relevance(self, question: str, document: str) -> float:
        """
        동적 의료 엔티티 추출을 통한 관련성 계산 (간소화된 출력)
        """
        # 엔티티 추출
        question_entities = self.extract_medical_entities(question)
        document_entities = self.extract_medical_entities(document)

        # 엔티티 매칭 점수 계산
        total_score = 0
        category_count = 0
        matched_categories = []

        for category in question_entities:
            if category in document_entities:
                q_entities = set(question_entities[category])
                d_entities = set(document_entities[category])
                common_entities = q_entities.intersection(d_entities)

                if q_entities:
                    category_score = len(common_entities) / len(q_entities)
                    total_score += category_score
                    category_count += 1
                    if common_entities:
                        matched_categories.append(f"{category}: {list(common_entities)[:2]}")

        entity_relevance = total_score / category_count if category_count > 0 else 0

        # 다국어 키워드 매칭
        question_lower = question.lower()
        document_lower = document.lower()

        keyword_matches = 0
        matched_keywords = []

        # 핵심 키워드들만 확인
        core_keywords = ['고혈압', 'hypertension', '당뇨병', 'diabetes', '폐렴', 'pneumonia',
                        '치료', 'treatment', '약물', 'medication', '증상', 'symptom',
                        'blood pressure', '혈압', '1차', 'first-line', 'primary']

        for keyword in core_keywords:
            if keyword in question_lower and keyword in document_lower:
                keyword_matches += 1
                matched_keywords.append(keyword)

        keyword_relevance = keyword_matches / len(core_keywords)

        # 의료 문맥 점수
        context_patterns = [
            'patient', 'treatment', 'therapy', 'medication', 'diagnosis',
            '환자', '치료', '약물', '진단', '처방'
        ]

        question_context = sum(1 for pattern in context_patterns if pattern in question_lower)
        document_context = sum(1 for pattern in context_patterns if pattern in document_lower)

        context_bonus = 0
        if question_context > 0 and document_context > 0:
            context_bonus = min(question_context, document_context) / len(context_patterns)

        # 최종 점수
        final_score = (entity_relevance * 0.4) + (keyword_relevance * 0.4) + (context_bonus * 0.2)

        # 간단한 디버깅 정보
        if final_score > 0:
            print(f"      매칭: 엔티티({entity_relevance:.3f}) + 키워드({keyword_relevance:.3f}) + 문맥({context_bonus:.3f}) = {final_score:.3f}")
            if matched_keywords:
                print(f"      키워드 매칭: {matched_keywords[:3]}")

        return min(final_score, 1.0)

    def enhanced_search(self, question: str, df: pd.DataFrame, top_k: int = 5) -> List[Dict]:
        """
        향상된 검색 기능 (다국어 지원)
        """
        # 질문 전처리
        processed_question = self.preprocess_question(question)

        # 임베딩 계산
        question_embedding = self.model.encode([processed_question])

        # 문서 임베딩 (캐싱 가능)
        if not hasattr(self, '_document_embeddings'):
            if 'context' in df.columns:
                documents = df['context'].fillna('').tolist()
            else:
                documents = []
                for _, row in df.iterrows():
                    question_text = row.get('question', '')
                    answer_text = row.get('answer', '')
                    combined_text = f"{question_text} {answer_text}"
                    documents.append(combined_text)

            print(f"문서 임베딩 생성 중... (총 {len(documents)}개)")
            self._document_embeddings = self.model.encode(documents)
            print("임베딩 완료")

        # 코사인 유사도 계산
        similarities = cosine_similarity(question_embedding, self._document_embeddings)[0]
        print(f"유사도 범위: {similarities.min():.3f} ~ {similarities.max():.3f}")

        # 의료 도메인 특화 유사도 계산
        medical_relevances = []
        print("의료 관련성 분석 중...")

        for idx, row in df.iterrows():
            if 'context' in df.columns:
                document_text = row.get('context', '')
            else:
                document_text = f"{row.get('question', '')} {row.get('answer', '')}"

            # 첫 번째 문서만 상세 분석
            if idx == 0:
                print("  첫 번째 문서 상세 분석:")
                med_relevance = self.calculate_medical_relevance(question, document_text)
            else:
                med_relevance = self.calculate_medical_relevance_silent(question, document_text)

            medical_relevances.append(med_relevance)

            # 처음 몇 개만 처리
            if idx >= 9:  # 10개만 처리
                break

        # 나머지는 0으로 채움
        while len(medical_relevances) < len(df):
            medical_relevances.append(0.0)

        medical_relevances = np.array(medical_relevances)
        print(f"의료 관련성 범위: {medical_relevances.min():.3f} ~ {medical_relevances.max():.3f}")

        # 종합 점수 계산 (코사인 유사도 60% + 의료 관련성 40%)
        combined_scores = similarities * 0.6 + medical_relevances * 0.4
        print(f"종합 점수 범위: {combined_scores.min():.3f} ~ {combined_scores.max():.3f}")

        # 상위 결과 선택
        top_indices = np.argsort(combined_scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            if combined_scores[idx] > 0.05:
                if 'context' in df.columns:
                    content = df.iloc[idx].get('context', '')
                else:
                    content = f"{df.iloc[idx].get('question', '')} {df.iloc[idx].get('answer', '')}"

                results.append({
                    'index': int(idx),
                    'similarity': float(combined_scores[idx]),
                    'content': content,
                    'question_original': df.iloc[idx].get('question', ''),
                    'answer': df.iloc[idx].get('answer', ''),
                    'category': df.iloc[idx].get('category', 'general')
                })

        print(f"최종 검색 결과: {len(results)}개")
        return results

    def calculate_confidence(self, search_results: List[Dict], question: str) -> float:
        """
        신뢰도 계산 개선 (의료 특화)
        """
        if not search_results:
            return 0.0

        scores = []

        # 1. 평균 유사도 점수 (가중치 증가)
        avg_similarity = np.mean([result['similarity'] for result in search_results])
        scores.append(avg_similarity)

        # 2. 최고 점수 기준 (상위 결과 품질)
        max_similarity = max(result['similarity'] for result in search_results)
        scores.append(max_similarity)

        # 3. 결과 수 기반 점수
        result_count_score = min(len(search_results) / 3.0, 1.0)  # 3개 이상이면 최고점
        scores.append(result_count_score)

        # 4. 의료 전문성 점수 (동적 계산)
        medical_score = 0
        for result in search_results:
            content = result['content']
            entities = self.extract_medical_entities(content)
            entity_count = sum(len(entity_list) for entity_list in entities.values())
            medical_score += min(entity_count / 5.0, 1.0)  # 5개 이상이면 만점

        medical_score = medical_score / len(search_results) if search_results else 0
        scores.append(medical_score)

        # 5. 의료 관련성 보너스 (새로 추가)
        question_lower = question.lower()
        has_medical_intent = any(keyword in question_lower for keyword in
                               ['고혈압', '당뇨병', '폐렴', '치료', '약물', '증상', '진단'])

        if has_medical_intent and max_similarity > 0.15:
            medical_bonus = min(max_similarity * 2, 0.3)  # 최대 0.3 보너스
        else:
            medical_bonus = 0

        scores.append(medical_bonus)

        # 가중 평균으로 최종 신뢰도 계산 (의료 관련성을 더 중시)
        weights = [0.25, 0.3, 0.15, 0.2, 0.1]  # 최고점수와 의료전문성을 중시
        final_confidence = sum(score * weight for score, weight in zip(scores, weights))

        return min(final_confidence, 1.0)

    def generate_response(self, question: str, df: pd.DataFrame) -> Dict[str, Any]:
        """
        개선된 응답 생성
        """
        # 검색 수행
        search_results = self.enhanced_search(question, df, top_k=5)

        # 신뢰도 계산
        confidence = self.calculate_confidence(search_results, question)

        # 답변 생성 로직
        if confidence < 0.5:
            answer = "검색된 정보의 신뢰도가 낮습니다. 의료 전문가와 상담하시기 바랍니다."
            recommendation = "질문을 더 구체적으로 작성하거나, 의료 용어를 포함해 주세요."
        else:
            # 상위 결과들로부터 답변 구성
            best_results = search_results[:3]
            answer_parts = []

            for result in best_results:
                if result['similarity'] > 0.15:  # 임계값 낮춤
                    answer_parts.append(result['answer'])

            if answer_parts:
                answer = " ".join(set(answer_parts))  # 중복 제거
            else:
                answer = "관련 정보를 찾았지만 정확한 답변을 제공하기 어렵습니다."

            recommendation = "추가 정보가 필요하시면 의료 전문가와 상담하세요."

        return {
            'question': question,
            'answer': answer,
            'confidence': confidence,
            'search_results_count': len(search_results),
            'sources': [
                {
                    'title': f"의료 문서 #{result['index']}",
                    'similarity': result['similarity'],
                    'category': result['category']
                }
                for result in search_results[:3]
            ],
            'recommendation': recommendation
        }

def display_improved_response(response_data: Dict[str, Any]):
    """
    개선된 응답 결과를 보기 좋게 출력 (이모티콘 제거)
    """
    print("="*80)
    print(" 개선된 의료 AI 어시스턴트 응답")
    print("="*80)
    print(f" 질문: {response_data['question']}")

    confidence = response_data['confidence']
    if confidence > 0.7:
        status = "[높음]"
    elif confidence > 0.5:
        status = "[보통]"
    else:
        status = "[낮음]"

    print(f" 신뢰도: {confidence:.3f} {status}")
    print(f" 검색 결과 수: {response_data['search_results_count']}")
    print("\n" + "-"*50)
    print(" 답변:")
    print("-"*50)
    print(response_data['answer'])

    if response_data['sources']:
        print("\n" + "-"*50)
        print(" 참조 자료:")
        print("-"*50)
        for i, source in enumerate(response_data['sources'], 1):
            if source['similarity'] > 0.7:
                confidence_level = "[높음]"
            elif source['similarity'] > 0.5:
                confidence_level = "[보통]"
            else:
                confidence_level = "[낮음]"
            print(f"{i}. {source['title']} {confidence_level}")
            print(f"   분야: {source['category']}, 유사도: {source['similarity']:.3f}")

    print(f"\n권장사항: {response_data['recommendation']}")
    print("="*80)
    print()

def suggest_better_questions(original_question: str, rag_system) -> List[str]:
    """
    동적 의료 엔티티 분석을 통한 질문 개선 제안
    """
    suggestions = []

    # 원본 질문에서 의료 엔티티 추출
    entities = rag_system.extract_medical_entities(original_question)

    # 질문의 구체성 분석
    question_lower = original_question.lower()

    # 너무 일반적인 질문인지 체크
    general_patterns = ['무엇', '어떤', '언제', '어떻게', '왜']
    is_general = any(pattern in question_lower for pattern in general_patterns)

    # 의료 엔티티가 있는 경우 더 구체적인 질문 제안
    if entities and is_general:
        # 질병 관련 엔티티가 있으면
        if 'disease_terms' in entities and entities['disease_terms']:
            disease = entities['disease_terms'][0]
            suggestions.extend([
                f"{disease}의 초기 증상과 진단 기준은 무엇인가요?",
                f"{disease} 환자의 생활 관리법은 어떻게 되나요?",
                f"{disease}의 합병증 예방법은 무엇인가요?"
            ])

        # 약물 관련 엔티티가 있으면
        if 'medication_terms' in entities and entities['medication_terms']:
            medication = entities['medication_terms'][0]
            suggestions.extend([
                f"{medication}의 정확한 복용법과 주의사항은?",
                f"{medication}의 부작용과 대처법은 무엇인가요?",
                f"{medication}와 상호작용하는 다른 약물은?"
            ])

        # 증상 관련 엔티티가 있으면
        if 'symptom_terms' in entities and entities['symptom_terms']:
            symptom = entities['symptom_terms'][0]
            suggestions.extend([
                f"{symptom} 증상이 나타나는 주요 질환은?",
                f"{symptom}이 지속될 때 응급 상황 판단 기준은?",
                f"{symptom} 완화를 위한 즉시 조치법은?"
            ])

    # 일반적인 개선 제안 (엔티티가 없거나 매우 모호한 경우)
    elif is_general:
        suggestions.extend([
            "구체적인 질병명, 약물명, 또는 증상을 포함해 주세요",
            "환자의 나이, 성별, 기존 병력 등 추가 정보를 제공해 주세요",
            "궁금한 점을 더 구체적으로 표현해 주세요 (예: 용량, 기간, 방법 등)"
        ])

    # 중복 제거하고 최대 3개만 반환
    unique_suggestions = list(dict.fromkeys(suggestions))
    return unique_suggestions[:3]

def run_improved_test(medical_df: pd.DataFrame):
    """
    개선된 테스트 실행 (실제 데이터 사용)
    """
    # 개선된 RAG 시스템 초기화
    rag_system = ImprovedMedicalRAG()

    # 테스트 질문들
    test_questions = [
        "고혈압 치료의 1차 약물은 무엇인가요?",
        "당뇨병의 주요 증상은 무엇인가요?",
        "폐렴 치료에 사용되는 항생제는?"
    ]

    print("개선된 의료 RAG 시스템 테스트")
    print(f"실제 의료 데이터: {len(medical_df)}개")
    print()

    for i, question in enumerate(test_questions, 1):
        print(f"테스트 {i}/{len(test_questions)}")

        # 응답 생성
        response = rag_system.generate_response(question, medical_df)
        display_improved_response(response)

        # 신뢰도가 낮으면 개선된 질문 제안
        if response['confidence'] < 0.6:
            suggestions = suggest_better_questions(question, rag_system)
            if suggestions:
                print("더 나은 결과를 위한 제안:")
                for j, suggestion in enumerate(suggestions[:2], 1):
                    print(f"  {j}. {suggestion}")
                print()

        time.sleep(1)

    print("개선된 테스트 완료!")

# 사용 예시
run_improved_test(medqa_df)

if __name__ == "__main__":
    print("개선된 의료 RAG 시스템이 준비되었습니다.")
    print("사용법: run_improved_test(medqa_df)")
    print("주의: medqa_df를 실제 데이터프레임 변수명으로 변경하세요.")

개선된 의료 RAG 시스템 테스트
실제 의료 데이터: 500개

테스트 1/3
문서 임베딩 생성 중... (총 500개)
임베딩 완료
유사도 범위: -0.150 ~ 0.235
의료 관련성 분석 중...
  첫 번째 문서 상세 분석:
      매칭: 엔티티(0.000) + 키워드(0.000) + 문맥(0.200) = 0.040
의료 관련성 범위: 0.000 ~ 0.040
종합 점수 범위: -0.090 ~ 0.141
최종 검색 결과: 5개
 개선된 의료 AI 어시스턴트 응답
 질문: 고혈압 치료의 1차 약물은 무엇인가요?
 신뢰도: 0.350 [낮음]
 검색 결과 수: 5

--------------------------------------------------
 답변:
--------------------------------------------------
검색된 정보의 신뢰도가 낮습니다. 의료 전문가와 상담하시기 바랍니다.

--------------------------------------------------
 참조 자료:
--------------------------------------------------
1. 의료 문서 #59 [낮음]
   분야: Medical Knowledge, 유사도: 0.141
2. 의료 문서 #282 [낮음]
   분야: Medical Knowledge, 유사도: 0.116
3. 의료 문서 #472 [낮음]
   분야: Medical Knowledge, 유사도: 0.113

권장사항: 질문을 더 구체적으로 작성하거나, 의료 용어를 포함해 주세요.

더 나은 결과를 위한 제안:
  1. 약의 정확한 복용법과 주의사항은?
  2. 약의 부작용과 대처법은 무엇인가요?

테스트 2/3
유사도 범위: -0.138 ~ 0.240
의료 관련성 분석 중...
  첫 번째 문서 상세 분석:
의료 관련성 범위: 0.000 ~ 0.013
종합 점수 범위: -0.083 ~ 0.144
최종 검색 결과: 5개
 개선된 의료 AI 어시스턴트 

## 8. 인터랙티브 의료 챗봇

In [51]:
def run_medical_chatbot(df: pd.DataFrame):
    """
    인터랙티브 의료 챗봇 실행
    """
    print(" 의료 AI 어시스턴트에 오신 것을 환영합니다!")
    print(" 의료 관련 질문을 자유롭게 해보세요.")
    print("⚠️  이 AI는 일반적인 의학 정보만 제공하며, 실제 의료 상담을 대체하지 않습니다.")
    print(" 종료하려면 'exit', 'quit', 또는 '종료'를 입력하세요.")
    print("-" * 80)

    conversation_history = []

    while True:
        try:
            # 사용자 입력
            user_input = input("\n👤 질문: ").strip()

            # 종료 조건
            if user_input.lower() in ['exit', 'quit', '종료', 'q']:
                print("\n 의료 AI 어시스턴트를 이용해 주셔서 감사합니다.")
                break

            if not user_input:
                print("❓ 질문을 입력해 주세요.")
                continue

            print("\n🔍 검색 중...")

            # 응답 생성
            response = generate_medical_response(
                question=user_input,
                df=df,
                response_type="general"
            )

            # 응답 출력
            print(f"\n🤖 의료 AI (신뢰도: {response['confidence']:.3f}):")
            print("-" * 50)
            print(response['answer'])

            # 참조 자료 출력 (간단히)
            if response['sources']:
                print(f"\n 참조 자료: {len(response['sources'])}개")
                for i, source in enumerate(response['sources'][:2], 1):  # 상위 2개만
                    print(f"   {i}. {source['title']} ({source['category']})")

            # 대화 기록 저장
            conversation_history.append({
                "user": user_input,
                "assistant": response['answer'],
                "confidence": response['confidence']
            })

            print("\n" + "="*80)

        except KeyboardInterrupt:
            print("\n\n 대화가 중단되었습니다.")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}")
            print("🔄 다시 시도해 주세요.")


# 챗봇 실행
print(" 의료 챗봇을 시작합니다...")
print(" 주의: 코랩 환경에서는 input() 함수가 제한될 수 있습니다.")
print(" 로컬 환경에서 실행하시면 완전한 대화형 경험을 할 수 있습니다.")

# 코랩 환경을 고려한 간단한 테스트
sample_queries = [
    "고혈압이란 무엇인가요?",
    "우울증 환자의 수면장애는 어떻게 치료하나요?",
    "소아의 주기성 구토증 진단을 진단해주세요."
]

print("\n 샘플 질문으로 시스템을 테스트해보겠습니다:")
for query in sample_queries:
    print(f"\n👤 질문: {query}")
    response = generate_medical_response(query, medqa_df)
    print(f" 답변: {response['answer'][:200]}...")
    print(f" 신뢰도: {response['confidence']:.3f}")

# 실제 대화형 모드는 주석 처리 (코랩 환경 고려)
# run_medical_chatbot(medqa_df)

 의료 챗봇을 시작합니다...
 주의: 코랩 환경에서는 input() 함수가 제한될 수 있습니다.
 로컬 환경에서 실행하시면 완전한 대화형 경험을 할 수 있습니다.

 샘플 질문으로 시스템을 테스트해보겠습니다:

👤 질문: 고혈압이란 무엇인가요?
 의료 검색 수행: '고혈압이란 무엇인가요?'
✅ 검색 완료! 상위 3개 결과 반환
 답변: - 주요 답변: 고혈압은 혈압이 정상 범위를 초과하여 지속적으로 높은 상태를 의미합니다. 일반적으로 수축기 혈압이 140mmHg 이상이거나 이완기 혈압이 90mmHg 이상일 때 고혈압으로 진단됩니다. 고혈압은 심장병, 뇌졸중, 신장 질환 등 여러 건강 문제의 위험을 증가시킬 수 있습니다.
- 근거: [의료 정보 3] (신뢰도: 0.173)에서 고혈압 환자의...
 신뢰도: 0.183

👤 질문: 우울증 환자의 수면장애는 어떻게 치료하나요?
 의료 검색 수행: '우울증 환자의 수면장애는 어떻게 치료하나요?'
✅ 검색 완료! 상위 3개 결과 반환
 답변: - 주요 답변: 우울증 환자의 수면장애는 일반적으로 인지행동치료(CBT)와 약물치료(항우울제 또는 수면제)를 통해 치료할 수 있습니다. CBT는 수면 위생을 개선하고 부정적인 생각을 변화시키는 데 도움을 줄 수 있습니다. 약물치료는 환자의 증상에 따라 다르게 적용될 수 있습니다.
- 근거: 제공된 의료 지식 베이스에는 우울증과 관련된 수면장애에 대한 구체적...
 신뢰도: 0.288

👤 질문: 소아의 주기성 구토증 진단을 진단해주세요.
 의료 검색 수행: '소아의 주기성 구토증 진단을 진단해주세요.'
✅ 검색 완료! 상위 3개 결과 반환
 답변: - 주요 답변: 소아의 주기성 구토증(Periodic Vomiting Syndrome, PVS)은 반복적인 구토 에피소드가 발생하는 상태로, 주로 스트레스, 감정적 요인 또는 특정 음식과 관련이 있을 수 있습니다. 진단은 임상적 평가와 증상 이력을 바탕으로 이루어지며, 다른 원인을 배제하기 위한 추가 검사도 필요할 수 있습니다.
- 근

## 9. 결과 요약 및 다음 단계

In [52]:
def generate_system_report(df: pd.DataFrame):
    """
    시스템 성능 리포트 생성
    """
    print(" MedQA 기반 의료 RAG 시스템 - Phase 1 완료 보고서")
    print("="*80)

    # 데이터 통계
    print(" 데이터 통계:")
    print(f"  • 총 의료 문서 수: {len(df):,}개")
    print(f"  • 평균 토큰 수: {df['n_tokens'].mean():.1f}")
    print(f"  • 최대 토큰 수: {df['n_tokens'].max():,}")
    print(f"  • 데이터 출처: {df['source'].value_counts().to_dict()}")

    # 카테고리 분포
    print("\n 의료 분야 분포:")
    category_counts = df['category'].value_counts()
    for category, count in category_counts.head(10).items():
        print(f"  • {category}: {count}개")

    # 임베딩 정보
    if len(df) > 0 and 'embeddings' in df.columns:
        sample_embedding = df['embeddings'].iloc[0]
        if isinstance(sample_embedding, list):
            embedding_dim = len(sample_embedding)
        else:
            embedding_dim = "Unknown"
        print(f"\n 임베딩 정보:")
        print(f"  • 임베딩 차원: {embedding_dim}")
        print(f"  • 임베딩 모델: {embedding_model}")

    print("\n" + "="*80)

    # Phase 1 성과
    print(" Phase 1 달성 목표:")
    print("  1. ✓ MedQA 데이터 로딩 및 전처리 완료")
    print("  2. ✓ 벡터 검색 시스템 구축 완료")
    print("  3. ✓ 의료 특화 프롬프트 템플릿 개발 완료")
    print("  4. ✓ 인터랙티브 챗봇 프로토타입 완료")

    # 다음 단계 제안
    print("\n 다음 단계 (Phase 2) 권장사항:")
    print("  1. 하이브리드 검색 (BM25 + 벡터) 구현")
    print("  2. 의료 용어 동의어 매핑 시스템 추가")
    print("  3. 메타데이터 기반 필터링 고도화")
    print("  4. 검색 성능 평가 메트릭 도입")
    print("  5. 웹 인터페이스 개발 (Streamlit/Gradio)")

    print("\n 개선 아이디어:")
    print("  • 의료 분야별 전문 모델 활용")
    print("  • 실시간 의료 데이터베이스 연동")
    print("  • 다국어 지원 (한영 의료 용어)")
    print("  • 의료영상 분석 기능 추가")
    print("  • 환자 기록 연동 시스템")

    print("\n" + "="*80)
    print(" 의료 RAG 시스템 Phase 1 완료!")

# 최종 리포트 생성
generate_system_report(medqa_df)

# 데이터 저장 확인
print("\n💾 저장된 파일:")
if os.path.exists('medqa_embeddings.csv'):
    file_size = os.path.getsize('medqa_embeddings.csv') / 1024 / 1024  # MB
    print(f"  • medqa_embeddings.csv ({file_size:.1f} MB)")
else:
    print("  • 파일 저장 실패")

 MedQA 기반 의료 RAG 시스템 - Phase 1 완료 보고서
 데이터 통계:
  • 총 의료 문서 수: 500개
  • 평균 토큰 수: 210.6
  • 최대 토큰 수: 523
  • 데이터 출처: {'GBaker/MedQA-USMLE-4-options': 500}

 의료 분야 분포:
  • Medical Knowledge: 500개

 임베딩 정보:
  • 임베딩 차원: 1536
  • 임베딩 모델: text-embedding-3-small

 Phase 1 달성 목표:
  1. ✓ MedQA 데이터 로딩 및 전처리 완료
  2. ✓ 벡터 검색 시스템 구축 완료
  3. ✓ 의료 특화 프롬프트 템플릿 개발 완료
  4. ✓ 인터랙티브 챗봇 프로토타입 완료

 다음 단계 (Phase 2) 권장사항:
  1. 하이브리드 검색 (BM25 + 벡터) 구현
  2. 의료 용어 동의어 매핑 시스템 추가
  3. 메타데이터 기반 필터링 고도화
  4. 검색 성능 평가 메트릭 도입
  5. 웹 인터페이스 개발 (Streamlit/Gradio)

 개선 아이디어:
  • 의료 분야별 전문 모델 활용
  • 실시간 의료 데이터베이스 연동
  • 다국어 지원 (한영 의료 용어)
  • 의료영상 분석 기능 추가
  • 환자 기록 연동 시스템

 의료 RAG 시스템 Phase 1 완료!

💾 저장된 파일:
  • medqa_embeddings.csv (17.7 MB)
